**NOTE**: This is a Julia demo for Binder, it's been adapted from the Julia demo on the nbconvert website.

Binder is an open-source project and Julia support is early in development. If you'd like to improve Julia support in Binder, see our issue on adding Julia support here: https://github.com/jupyter/repo2docker/issues/23. We'd love to hear from you!

# An IJulia Demo

This notebook uses **IJulia**: a [Julia-language](http://julialang.org/) backend combined with the [IPython](http://ipython.org/) interactive environment.   This combination allows you to interact with the Julia language using IPython's powerful [graphical notebook](http://ipython.org/notebook.html), which combines code, formatted text, math, and multimedia in a single document.

## Basic Julia interaction

Basic mathematical expressions work like you expect:

In [1]:
1 + sin(3)

1.1411200080598671

You can define variables, write loops, and execute arbitrary multiline code blocks.  Here is an example of an alternating harmonic series $\sum_{n=1}^\infty \frac{(-1)^n}{n}$ from a [Julia tutorial by Homer Reid](http://homerreid.ath.cx/teaching/18.330/JuliaProgramming.shtml#SimplePrograms):

In [2]:
s = 0.0
for n = 1:2:10000
    s += 1/n - 1/(n+1)
end
s # an expression on the last line (if it doesn't end with ";") is printed as "Out"

0.6930971830599458

Previous outputs can be referred to via `Out[`*n*`]`, following the IPython, for example `Out[2]` for the result above.  You can also use the shorthand `_2`, or `_` for the previous result, as in IPython.  Like in Matlab, `ans` can also be used to refer to the previous result, *even if it was not printed* (when the command ended with `;`).

For example, the harmonic series above should be converging (slowly) to $\ln 2$, and we can check this:

In [3]:
Out[2] - log(2)

-4.9997499999454575e-5

Like Matlab or Scipy + Numpy, Julia has lots of mathematical functions and linear algebra built in.  For example, we can define a $500\times500$ random matrix $R$ and form the positive-definite matrix $R^* R$:

In [4]:
R = rand(500,500)
R' * R

500×500 Matrix{Float64}:
 159.321  119.096  125.809  119.44   …  116.299  112.048  117.386  125.247
 119.096  164.07   131.457  120.328     119.273  115.089  119.536  128.35
 125.809  131.457  174.423  131.492     129.098  122.37   124.978  130.889
 119.44   120.328  131.492  173.006     125.494  119.671  122.307  129.75
 122.11   120.754  128.367  127.28      119.276  114.847  120.632  123.719
 119.865  122.381  129.643  125.988  …  122.997  118.326  124.985  131.685
 120.935  120.552  128.811  129.747     120.43   119.337  122.357  127.619
 122.194  123.14   127.616  123.138     120.378  118.204  118.761  127.393
 117.19   116.679  125.864  117.769     115.333  111.998  114.307  121.104
 119.079  121.193  129.53   126.936     123.203  116.139  115.939  128.431
 123.071  126.35   133.529  125.883  …  124.116  119.417  120.128  129.506
 122.723  124.686  131.876  128.373     123.29   118.789  122.005  130.673
 117.726  123.846  130.171  129.253     124.048  119.766  120.997  130.032
  

(Notice that, by default, only a portion of a large matrix is shown.  You didn't really want to read $500^2 = 250,000$ numbers, did you?)

Standard output from Julia is also captured and sent to the IJulia notebook as you'd expect:

In [5]:
println("Hello world!\n")
println(stderr, "Börk börk börk, some unicode output on stderr...\n")

Hello world!



Börk börk börk, some unicode output on stderr...



IJulia even captures output from external C libraries (and notice how easy it is using Julia's `ccall` intrinsic):

In [6]:
ccall(:printf, Cint, (Ptr{UInt8},), "Hello from C!!\n");

Hello from C!!


We can define functions, of course, and use them in later input cells:

In [7]:
f(x) = x .+ 1

f (generic function with 1 method)

In [8]:
println(f(3))
f([1,1,2,3,5,8])

4


6-element Vector{Int64}:
 2
 2
 3
 4
 6
 9

Notice that the input above both printed an scalar to `STDOUT` and also returned a vector, the latter using Julia's ability to write polymorphic functions and built-in array operations.

On the other hand adding a string to a number is not defined (there is no `+` method defined for those types, although we could easily add one), and attempting to do so will throw an exception:

In [9]:
f("Hello?")

LoadError: MethodError: no method matching +(::String, ::Int64)
[0mClosest candidates are:
[0m  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:560
[0m  +([91m::T[39m, ::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} at int.jl:87
[0m  +([91m::LinearAlgebra.UniformScaling[39m, ::Number) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.6/LinearAlgebra/src/uniformscaling.jl:145
[0m  ...

# Interactive Plotting in IJulia

Below we'll show off some plotting using the excellent Gadfly package. The plots are heavily inspired by [this blog post](http://int8.io/basic-visualization-in-julia-gadfly/).

In [ ]:
using RDatasets
sleep = dataset("lme4","sleepstudy")

In [ ]:
using Gadfly
plot(sleep, x = "Days", y = "Reaction", Geom.point)

In [ ]:
plot(sleep, x = "Reaction", Geom.density, color = "Subject", Scale.x_continuous(minvalue= 0, maxvalue= 500))

In [ ]:
plot(x = sleep[!, :Reaction], Geom.histogram(bincount = 30), Scale.x_continuous(minvalue = 200), color = sleep[!, :Days])

## Multimedia display in IJulia

Like most programming languages, Julia has a built-in `print(x)` function for outputting an object `x` as text, and you can override the resulting text representation of a user-defined type by overloading Julia's `show` function.  The next version of Julia, however, will extend this to a more general mechanism to display **arbitrary multimedia representations** of objects, as defined by standard [MIME types](https://en.wikipedia.org/wiki/Internet_media_type).   More specifically, the Julia [multimedia I/O API](http://docs.julialang.org/en/latest/stdlib/base/#multimedia-i-o) provides:

* A `display(x)` function requests the richest available multimedia display of a Julia object x (with a `text/plain` fallback).
* Overloading `writemime` allows one to indicate arbitrary multimedia representations (keyed by standard MIME types) of user-defined types.
* Multimedia-capable display backends may be registered by subclassing a generic `Display` type.  IJulia provides one such backend which, thanks to the IPython notebook, is capable of displaying HTML, LaTeX, SVG, PNG, and JPEG media formats.

The last two points are critical, because they separate **multimedia export** (which is defined by functions associated with the originating Julia data) from **multimedia display** (defined by backends which know nothing about the source of the data).

Precisely these mechanism were used to create the inline PyPlot plots above.   To start with, the simplest thing is to provide the MIME type of the data when you call `display`, which allows you to pass "raw" data in the corresponding format:

In [ ]:
display("text/html", """Hello <b>world</b> in <font color="red">HTML</font>!""")

However, it will be more common to attach this information to types, so that they display correctly automatically.  For example, let's define a simple `HTML` type in Julia that contains a string and automatically displays as HTML (given an HTML-capable backend such as IJulia):

In [ ]:
mutable struct HTML
   s::String
end

Base.show(io::IO, ::MIME"text/html", x::HTML) = print(io, x.s)

Here, `Base.show` is just a function that writes `x` in the corresponding format (`text/html`) to the I/O stream `io`.

This `show` definition is all that we need to make any object of type `HTML` display automatically as HTML text in IJulia:

In [ ]:
x = HTML("<ul> <li> Hello from a bulleted list! </ul>")

In [ ]:
display(x)
println(x)